In [1]:
import hopsworks
import pandas as pd

In [2]:
project = hopsworks.login(
    host="2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai",
    project="electricity",
    api_key_value="xIGaDiwBZpXxLYRQ.SgheZwqHTZteASnO1PtPMC4HMgco2OVD6VK6xZN3sKCy96FQT27rF0JbBuFivTTA",
)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287


Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Read local csv file containing holidays
se_holidays_pdf = pd.read_csv("data/holidays.csv")

# Convert day column to datetime
se_holidays_pdf['zone'] = "SE3"
se_holidays_pdf['day'] = pd.to_datetime(se_holidays_pdf['day'], format='%Y-%m-%d')

# Replace strings with 1 if holiday and drop original column
se_holidays_pdf['is_holiday'] = se_holidays_pdf['type_of_day'].map(lambda x: 1 if x == "holiday" else 0)
se_holidays_pdf.drop('type_of_day', inplace=True, axis=1)

In [7]:
se_holidays_pdf.dtypes

day           datetime64[ns]
zone                  object
is_holiday             int64
dtype: object

In [14]:
se_holidays_fg = fs.get_or_create_feature_group(
    name="se_holidays",
    version=1,
    description="Sweden red days",
    primary_key=["zone"],
    event_time="day",
    online_enabled=True,
    statistics_config={
        'histograms': True, 
        'correlations': True
    }
)

time travel format: HUDI


In [15]:
se_holidays_fg.insert(se_holidays_pdf)

time travel format: HUDI
Feature Group created successfully, explore it at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fg/14351


Uploading Dataframe: 0.00% |          | Rows 0/1095 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai/p/7287/jobs/named/se_holidays_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f96112f6b80>, None)